<a href="https://colab.research.google.com/github/Elena-Gauci/Hornet-Detection/blob/main/YOLOv8_vs_YOLOV11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf Hornet-Detection
!git clone https://github.com/Elena-Gauci/Hornet-Detection.git
%cd Hornet-Detection
!ls

Cloning into 'Hornet-Detection'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 65.97 KiB | 3.14 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/Hornet-Detection
Preprocessing.ipynb  README.md	yolov8_train_colab.ipynb


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -q ultralytics

In [4]:
from ultralytics import YOLO
from sklearn.model_selection import KFold
from pathlib import Path
import yaml
import numpy as np
import itertools
import random
import json
import os

In [5]:
from pathlib import Path

data_yaml = Path("data.yaml")

data_yaml.write_text(
"""path: /content/drive/MyDrive/hornet_detection/Occlusion-Test-5-bbox
train: train/images
val: valid/images

nc: 3
names:
  0: bee
  1: hornet
  2: other
"""
)

152

In [6]:
!ls /content/drive/MyDrive/hornet_detection/Occlusion-Test-5-bbox/train/images | head

frame_000037_jpg.rf.ab07d011e57481dbe1241e62d1815541.jpg
frame_000039_jpg.rf.6219cc9094cfad8ef4023bb07040f61c.jpg
frame_000040_jpg.rf.3c9f27187f3a48e8fda7817b0e640467.jpg
frame_000042_jpg.rf.668a9f74e32d6a82cb551047f7e73ce8.jpg
frame_000043_jpg.rf.d66393e5e044dadcdaed6b077ef85a80.jpg
frame_000044_jpg.rf.4e24bf18e33992b81038246cc769591f.jpg
frame_000045_jpg.rf.cf83e1bd935c2adfd7414f81f61619ff.jpg
frame_000046_jpg.rf.fb8f8b7867d3a8f31192f1c987e18143.jpg
frame_000047_jpg.rf.dbd7420042f6676204e6e56ef6702c11.jpg
frame_000048_jpg.rf.5dc52363f54bafbddd41093f0998a375.jpg


In [7]:
from pathlib import Path
print("CWD:", Path.cwd())
print("data.yaml exists here:", Path("data.yaml").resolve())
print(Path("data.yaml").read_text())

CWD: /content/Hornet-Detection
data.yaml exists here: /content/Hornet-Detection/data.yaml
path: /content/drive/MyDrive/hornet_detection/Occlusion-Test-5-bbox
train: train/images
val: valid/images

nc: 3
names:
  0: bee
  1: hornet
  2: other



In [15]:
BASE_DATA_YAML = str(Path("data.yaml").resolve())

with open(BASE_DATA_YAML) as f:
    base_data = yaml.safe_load(f)

DATA_ROOT = Path(base_data["path"])
ALL_IMAGES_DIR = DATA_ROOT / "train/images"

ALL_IMAGES = sorted(ALL_IMAGES_DIR.glob("*.jpg"))
assert len(ALL_IMAGES) > 0, "No images found"

In [16]:
N_FOLDS = 5
N_TRIALS = 5              # deliberately small (critical)
IMG_SIZE = 1024
EPOCHS = 200
PATIENCE = 30
SEED = 42

MODELS = ["yolov8n.pt", "yolov11n.pt"]

In [17]:
SEARCH_SPACE = {
    "lr0": [1e-4, 3e-4, 1e-3], #learning rate
    "batch": [4, 8], # batch size
    "weight_decay": [1e-5, 1e-4, 5e-4], # L2 regularisation applied to the model’s weights during optimisation, it adds a penalty term to the loss
    "mosaic": [0.5], # mosaic is a YOLO-specific augmentation that stitches 4 different images into one. mosaic is applied to 50% and 100% of the images. Since most images have only one hornet, this could lead to having the generated images with no hornets at all. So maybe remove the 1 (100%) option.
    "scale": [1] #scales the images. This may not be relevant in our case and we should stick to one, since the model memorising the background/ beehive/ beehive entrance is actually beneficial as the images have constant background. It may be beneficial to generalise hornets far away vs hornets very close to the camera.
}

In [18]:
def sample_params(space):
    return {k: random.choice(v) for k, v in space.items()}

In [22]:
# create a yaml for k-fold cross validation

def write_fold_yaml(fold_id, train_imgs, val_imgs):
    fold_dir = Path(f"fold_{fold_id}")
    fold_dir.mkdir(exist_ok=True)

    train_txt = fold_dir / "train.txt"
    val_txt = fold_dir / "val.txt"

    train_txt.write_text("\n".join(str(p.resolve()) for p in train_imgs))
    val_txt.write_text("\n".join(str(p.resolve()) for p in val_imgs))

    data = base_data.copy()

    # CRITICAL FIX: remove 'path'
    data.pop("path", None)

    data["train"] = str(train_txt.resolve())
    data["val"] = str(val_txt.resolve())

    fold_yaml = fold_dir / "data.yaml"
    with open(fold_yaml, "w") as f:
        yaml.safe_dump(data, f)

    return fold_yaml




In [ ]:
def format_run_name(model_name, params):
    lr = params["lr0"]
    batch = params["batch"]
    wd = params["weight_decay"]

    lr_str = f"{lr:.0e}".replace("-", "")
    wd_str = f"{wd:.0e}".replace("-", "")

    return f"{model_name}_{lr_str}_{batch}_{wd_str}"

In [23]:
DRIVE_RUNS_DIR = "/content/drive/MyDrive/hornet_detection/occlusion-test/runs"

def train_eval(model_name, fold_yaml, params):
    model = YOLO(model_name)

    run_name = format_run_name(model_name, params)

    run_dir = f"{DRIVE_RUNS_DIR}/{run_name}"
    last_ckpt = f"{run_dir}/weights/last.pt"

    resume = os.path.exists(last_ckpt)

    if resume:
        print(f"Resuming training from {last_ckpt}")
    else:
        print(f"Starting fresh run: {run_name}")

    model.train(
        data=fold_yaml,
        imgsz=IMG_SIZE,
        epochs=EPOCHS,
        patience=PATIENCE,
        optimizer="AdamW",
        seed=SEED,
        verbose=False,

        project=DRIVE_RUNS_DIR,
        name=run_name,
        exist_ok=True,
        resume=resume,

        **params
    )

    metrics = model.val(
        data=fold_yaml,
        project=DRIVE_RUNS_DIR,
        name=run_name,
        verbose=False
    )

    return metrics.box.map50


In [ ]:
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

results = {m: [] for m in MODELS}

for model_name in MODELS:
    print(f"\n=== Evaluating {model_name} ===")

    for fold_id, (train_idx, val_idx) in enumerate(kf.split(ALL_IMAGES)):
        print(f"\nFold {fold_id + 1}/{N_FOLDS}")

        train_imgs = [ALL_IMAGES[i] for i in train_idx]
        val_imgs = [ALL_IMAGES[i] for i in val_idx]

        fold_yaml = write_fold_yaml(fold_id, train_imgs, val_imgs)

        best_score = -1
        best_params = None

        for trial in range(N_TRIALS):
            params = sample_params(SEARCH_SPACE)
            print(f" Trial {trial + 1}/{N_TRIALS}: {params}")

            score = train_eval(model_name, fold_yaml, params)

            if score > best_score:
                best_score = score
                best_params = params

        print(f" Best mAP50: {best_score:.4f}")
        results[model_name].append(best_score)


Streaming output truncated to the last 5000 lines.
    141/200       3.3G       1.49      1.161     0.8809          6       1024: 100% ━━━━━━━━━━━━ 18/18 2.5it/s 7.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 4.3it/s 0.7s
                   all         17        546      0.941      0.541      0.593      0.334

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    142/200       3.3G      1.448      1.106     0.8626         14       1024: 100% ━━━━━━━━━━━━ 18/18 2.9it/s 6.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 3.6it/s 0.8s
                   all         17        546      0.945       0.54      0.594      0.332

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    143/200       3.3G      1.589      1.109     0.8601         17       1024: 100% ━━━━━━━━━━━━ 18/18 2.8it/s 6.5s
               

In [ ]:
print("\n=== FINAL RESULTS ===")

for model_name, scores in results.items():
    print(
        f"{model_name}: "
        f"mAP50 = {np.mean(scores):.4f} ± {np.std(scores):.4f}"
    )
